In [1]:
using ConcurrentSim

# Define the simulation components
mutable struct Chef
    id::Int
    station::Symbol
end

mutable struct Dish
    id::Int
    name::String
    prep_time::Float64
    cooking_time::Float64
    plating_time::Float64
end

mutable struct Order
    id::Int
    dishes::Vector{Dish}
end

# Define the simulation processes
function process_dish(env::Environment, chef::Chef, dish::Dish)
    @process env begin
        println("Chef $(chef.id) preparing $(dish.name) (Dish $(dish.id)) at $(now(env))")
        sleep(env, dish.prep_time)
        println("Chef $(chef.id) finished preparing $(dish.name) (Dish $(dish.id)) at $(now(env))")

        println("Chef $(chef.id) cooking $(dish.name) (Dish $(dish.id)) at $(now(env))")
        sleep(env, dish.cooking_time)
        println("Chef $(chef.id) finished cooking $(dish.name) (Dish $(dish.id)) at $(now(env))")

        println("Chef $(chef.id) plating $(dish.name) (Dish $(dish.id)) at $(now(env))")
        sleep(env, dish.plating_time)
        println("Chef $(chef.id) finished plating $(dish.name) (Dish $(dish.id)) at $(now(env))")
    end
end

function order_process(env::Environment, order::Order, chefs::Vector{Chef})
    for dish in order.dishes
        chef = rand(chefs)
        wait(env, process_dish(env, chef, dish))
    end

    @process env begin
        println("Order $(order.id) completed at $(now(env))")
    end
end

# Set up the simulation
function run_simulation(num_orders, num_chefs)
    env = Environment()
    chefs = [Chef(i, :prep) for i in 1:num_chefs]

    dish_names = [
        "Sizzling Steak Surprise",
        "Creamy Mushroom Medley",
        "Spicy Shrimp Spectacular",
        "Crispy Chicken Craze",
        "Tangy Tomato Twist",
        "Cheesy Broccoli Blast",
        "Garlicky Potato Perfection",
        "Zesty Zucchini Zinger",
        "Savory Spinach Sensation",
        "Buttery Bacon Bliss"
    ]

    for i in 1:num_orders
        dishes = [Dish(j, rand(dish_names), rand(1:5), rand(5:10), rand(1:3)) for j in 1:rand(1:3)]
        order = Order(i, dishes)
        @process env begin
            println("Order $(order.id) started at $(now(env))")
            order_process(env, order, chefs)
        end
    end

    run(env)
end

# Run the simulation
run_simulation(10, 3)